In [1]:
from cobra.io import read_sbml_model
#import os

## Maximum yield and productivity using default medium
### 1. Using model yeast_resv_glc with glucose as carbon source

In [2]:
# ONLY  OUTCOMENT THESE LINES IF NEEDED AND COMMENT THEM AGAIN AFTERWARDS
# Verify working directory and change it if needed
#os.getcwd()
#os.chdir("/Users/abril/Documents/DTU/CFD_Resveratrol/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae")

In [3]:
model = read_sbml_model("../data/models/yeast_resv_glc.xml")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [4]:
# Model olverview
model

Name,iMM904
Memory address,0x011add1000
Number of metabolites,1229
Number of reactions,1581
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


### Calculate theoretical maximum yields for chosen product for suitable carbon sources

In [5]:
# Verify if there is already an objective function (model exploration)
print(model.objective)

Maximize
1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090


In [6]:
# The present objective function belongs to:
model.reactions.BIOMASS_SC5_notrace

Reaction identifier,BIOMASS_SC5_notrace
Name,Biomass SC5 notrace
Memory address,0x11c387160
Stoichiometry,1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024... 1.1348 1 3 beta D Glucan C6H10O5 + 0.4588 L-Alanine + 0.046 AMP C10H12N5O7P + 0.1607 L-Arginine + 0.1017 L-Asparagine + 0.2975 L-Aspartate + 59.276 ATP C10H12N5O13P3 + 0.0447 CMP C9H12N3O8P +...
GPR,
Lower bound,0.0
Upper bound,999999.0


In [7]:
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x11c429990
Stoichiometry,e4_coumaroyl_coa + 3.0 h_c + 3.0 malcoa_c --> 4.0 co2_c + 4.0 coa_c + trans_resv + 3.0 H+ + 3.0 Malonyl CoA C24H33N7O19P3S --> 4.0 CO2 CO2 + 4.0 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,999999.0


##### Calculating maximum productivity and theoretical yield.
Using default medium concentration.

In [8]:
## Setting the objective reaction to resveratrol production.
with model:
    model.objective = model.reactions.VVVST1
    resv_production = model.optimize().objective_value
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3))

Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  1.179
Theoretical max. yield [mmol-glc / mmol-resv]: 0.118


### Maximum yield and productivity increasing carbon source.

In [9]:
# Get the medium composition concentration based on the exchange reactions.
for m in model.medium.items():
    print(model.reactions.get_by_id(m[0]), "|", model.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

EX_fe2_e: fe2_e <=>  | Fe2+ exchange | conc. = 999999.0
EX_glc__D_e: glc__D_e <=>  | D-Glucose exchange | conc. = 10.0
EX_h2o_e: h2o_e <=>  | H2O exchange | conc. = 999999.0
EX_h_e: h_e <=>  | H+ exchange | conc. = 999999.0
EX_k_e: k_e <=>  | K+ exchange | conc. = 999999.0
EX_na1_e: na1_e <=>  | Sodium exchange | conc. = 999999.0
EX_so4_e: so4_e <=>  | Sulfate exchange | conc. = 999999.0
EX_nh4_e: nh4_e <=>  | Ammonia exchange | conc. = 999999.0
EX_o2_e: o2_e <=>  | O2 exchange | conc. = 2.0
EX_pi_e: pi_e <=>  | Phosphate exchange | conc. = 999999.0


Inscreasing glucose concentration.

In [10]:
model.reactions.EX_glc__D_e.flux 

-10.0

In [11]:
#model.optimize().objective_value

In [33]:
# Growth rate with default glucose cocnentration
medium = model.medium
with model:
    growth_rate = model.optimize().objective_value                  #Growth rate
    glc_con = (-1*model.reactions.EX_glc__D_e.flux)
    print("Glucose concentration is:", round(glc_con,3))
    print("Maximum growth rate is  [h^-1] = ", round(growth_rate,3), "\n")

    # Increase glucose concentration.
    medium['EX_glc__D_e'] = 1646 #Glucose concentration to have the max growth rate.
    model.medium = medium

    #Re-calculate growth rate with new glucose concentration
    growth_rate = model.optimize().objective_value
    glc_con = (-1*model.reactions.EX_glc__D_e.flux)
    print("Glucose concentration is:", round(glc_con,3))
    print("Maximum growth rate is  [h^-1] = ",round(growth_rate,3))

Glucose concentration is: 10.0
Maximum growth rate is  [h^-1] =  0.288 

Glucose concentration is: 1646.0
Maximum growth rate is  [h^-1] =  35.881


We performed this simulation to know the limits of our system. According to the results, a glucose concentration of 1646.00 results in the maximum cell growth rate. However this does not mean it is the ideal glucose concentration. To make assumptions it it necessary to also observ the effects increasing the glucose concentration has on resveratrol theoretical productivity and yield.

##### Now lets see the effects of increasing glucose concentration on resveratrol yield and production.

In [41]:
medium = model.medium

with model:
    model.objective = model.reactions.VVVST1
    resv_production = model.optimize().objective_value
    print("With glucose concentration", (-1*model.reactions.EX_glc__D_e.flux), "\n")
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3), "\n")
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3), "\n \n")

    medium['EX_glc__D_e'] = 1646
    model.medium = medium

    resv_production = model.optimize().objective_value
    print("With glucose concentration",(-1*model.reactions.EX_glc__D_e.flux))
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3), "\n")
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3))

With glucose concentration 10.0 

Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  1.179 

Theoretical max. yield [mmol-glc / mmol-resv]: 0.118 
 

With glucose concentration 1646.0
Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  141.407 

Theoretical max. yield [mmol-glc / mmol-resv]: 0.086


As it can be seen, increasin the glucose conentration to achieve the maximum growth rate results in an maximum resveratrol productivity of 141.407 [mmol gDW^-1 h^-1] (much better than the the previous 1.179 [mmol gDW^-1 h^-1]).  

However, the theoretical maximum yield is reduced to 0.086 [mmol-glc / mmol-resv].

With this in mind, to find an optimal glucose concentration is needed.


##### Lets find an optimal glucose concentration.

In [ ]:
medium = model.medium

with model:
    model.objective = model.reactions.VVVST1
    resv_production = model.optimize().objective_value
    print("With glucose concentration", (-1*model.reactions.EX_glc__D_e.flux), "\n")
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3), "\n")
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3), "\n \n")

    medium['EX_glc__D_e'] = 1646
    model.medium = medium

    resv_production = model.optimize().objective_value
    print("With glucose concentration",(-1*model.reactions.EX_glc__D_e.flux))
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3), "\n")
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3))

### 2. Using model yeast_resv_cou with coumaric acid as carbon source

In [137]:
model_resv_cou = read_sbml_model("../data/models/yeast_resv_cou.xml")

In [138]:
model_resv_cou

Name,iMM904
Memory address,0x012eff3b20
Number of metabolites,1229
Number of reactions,1580
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [140]:
model_resv_cou.reactions.get_by_id('STS')

Reaction identifier,STS
Name,
Memory address,0x12fc57fa0
Stoichiometry,e4_coumaroyl_coa + 3.0 h_c + 3.0 malcoa_c --> 4.0 co2_c + 4.0 coa_c + trans_resv + 3.0 H+ + 3.0 Malonyl CoA C24H33N7O19P3S --> 4.0 CO2 CO2 + 4.0 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,999999.0


In [141]:
## Setting the objective reaction to resveratrol production
with model_resv_cou:
    model_resv_cou.objective = model_resv_cou.reactions.STS
    resv_production = model_resv_cou.optimize().objective_value
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ", resv_production)
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", resv_production / (-1*model_resv_cou.reactions.EX_glc__D_e.flux))

Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  7.13435570401729e-18
Theoretical max. yield [mmol-glc / mmol-resv]: 2.846310661081899e-17


In [142]:
# Get the medium composition concentration based on the exchange reactions.
for m in model_resv_cou.medium.items():
    print(model_resv_cou.reactions.get_by_id(m[0]), "|", model_resv_cou.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

EX_fe2_e: fe2_e <=>  | Fe2+ exchange | conc. = 999999.0
EX_glc__D_e: glc__D_e <=>  | D-Glucose exchange | conc. = 10.0
EX_h2o_e: h2o_e <=>  | H2O exchange | conc. = 999999.0
EX_h_e: h_e <=>  | H+ exchange | conc. = 999999.0
EX_k_e: k_e <=>  | K+ exchange | conc. = 999999.0
EX_na1_e: na1_e <=>  | Sodium exchange | conc. = 999999.0
EX_so4_e: so4_e <=>  | Sulfate exchange | conc. = 999999.0
EX_nh4_e: nh4_e <=>  | Ammonia exchange | conc. = 999999.0
EX_o2_e: o2_e <=>  | O2 exchange | conc. = 2.0
EX_pi_e: pi_e <=>  | Phosphate exchange | conc. = 999999.0


In [143]:
model_resv_cou.reactions

[<Reaction CITtcp at 0x12f0b2c80>,
 <Reaction 13BGH at 0x12f1896f0>,
 <Reaction 13BGHe at 0x12f189a50>,
 <Reaction 13GS at 0x12f18a140>,
 <Reaction 16GS at 0x12f18a680>,
 <Reaction 23CAPPD at 0x12f18ab00>,
 <Reaction 2DDA7Ptm at 0x12f18b9d0>,
 <Reaction 2DHPtm at 0x12f18a050>,
 <Reaction 2DOXG6PP at 0x12f18acb0>,
 <Reaction 2HBO at 0x12f18afe0>,
 <Reaction 2HBt2 at 0x12f18b490>,
 <Reaction 2HMHMBQMTm at 0x12f18afb0>,
 <Reaction 2HP6MPMOm at 0x12f18a710>,
 <Reaction 2HPMBQMTm at 0x12f1bdba0>,
 <Reaction 2HPMMBQMOm at 0x12f1be200>,
 <Reaction 2MBACt at 0x12f1be8f0>,
 <Reaction EX_epistest_SC_e at 0x12f18b460>,
 <Reaction EX_epist_e at 0x12f18b310>,
 <Reaction 2MBALDt at 0x12f18b190>,
 <Reaction 2MBALDtm at 0x12f18b610>,
 <Reaction 2MBTOHt at 0x12f1bf160>,
 <Reaction ACACT6p at 0x12f1bf310>,
 <Reaction 2MBTOHtm at 0x12f1bf4c0>,
 <Reaction ACACT7p at 0x12f1bf8e0>,
 <Reaction 2MPPALt at 0x12f1bf670>,
 <Reaction ACACT8p at 0x12f1bfe20>,
 <Reaction ACACT9p at 0x12f1bf3a0>,
 <Reaction ACALDCD 

In [144]:
# Recalculate resveratrol production with new coumaric acid concentration
medium = model_resv_cou.medium
with model_resv_cou:
    model_resv_cou.objective = model_resv_cou.reactions.STS
    resv_production = model_resv_cou.optimize().objective_value
    print("With glucose concentration",(-1*model_resv_cou.reactions.EX_glc__D_e.flux),", maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model_resv_cou.reactions.EX_glc__D_e.flux),3), "\n")

    medium['EX_glc__D_e'] = 100
    model_resv_cou.medium = medium

    resv_production = model_resv_cou.optimize().objective_value
    print("With glucose concentration",(-1*model_resv_cou.reactions.EX_glc__D_e.flux),", maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model_resv_cou.reactions.EX_glc__D_e.flux),3))

With glucose concentration 0.25065274151436023 , maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  0.0
Theoretical max. yield [mmol-glc / mmol-resv]: 0.0 

With glucose concentration 0.25065274151436023 , maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  0.0
Theoretical max. yield [mmol-glc / mmol-resv]: 0.0
